In [2]:
import os
os.chdir("D:\\github\\vn_backtester")
os.getcwd()

'D:\\github\\vn_backtester'

In [3]:
import pandas as pd
import numpy as np

__closepath__ = 'data/vn_stock/price_volume/close_matrix_top100_20120101-20240101.txt'
__openpath__ = 'data/vn_stock/price_volume/open_matrix_top100_20120101-20240101.txt'
__highpath__ = 'data/vn_stock/price_volume/high_matrix_top100_20120101-20240101.txt'
__lowpath__ = 'data/vn_stock/price_volume/low_matrix_top100_20120101-20240101.txt'
__volumepath = 'data/vn_stock/price_volume/volume_matrix_top100_20120101-20240101.txt'

def load_and_process_data(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df.set_index('time', inplace=True)
    df = df.astype(float)
    return df

close = load_and_process_data(__closepath__)
open = load_and_process_data(__openpath__)
high = load_and_process_data(__highpath__)
low = load_and_process_data(__lowpath__)
volume = load_and_process_data(__volumepath)

In [5]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score

def prepare_data(df, ticker, feature_windows=[9, 13, 21]):
    data = df[[ticker]].dropna()
    data['time_index'] = np.arange(len(data))
    
    for window in feature_windows:
        data[f'SMA_{window}'] = data[ticker].rolling(window=window).mean()  # Simple Moving Average
        data[f'EMA_{window}'] = data[ticker].ewm(span=window, adjust=False).mean()  # Exponential Moving Average
        data[f'Rolling_Var_{window}'] = data[ticker].rolling(window=window).var()  # Rolling Variance
        data[f'Rolling_Std_{window}'] = data[ticker].rolling(window=window).std()  # Rolling Std Dev
    
    data = data.fillna(0)

    # Add lag features
    data['Lag_1'] = data[ticker].shift(1)
    data['Lag_2'] = data[ticker].shift(2)
    data['Lag_3'] = data[ticker].shift(3)
    data = data.fillna(0)
    
    features = [col for col in data.columns if col != ticker]
    X = data[features]
    y = data[ticker]
    
    # Split the data before and after 2022
    X_train = X[X.index < '2022-01-01']
    y_train = y[X.index < '2022-01-01']
    
    X_test = X[X.index >= '2022-01-01']
    y_test = y[X.index >= '2022-01-01']
    
    return X_train, X_test, y_train, y_test

def tune_hyperparameters(X_train, y_train, model_type='ridge', cv_folds=5):
    if model_type == 'ridge':
        model = Ridge()
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]}
    else:
        model = LinearRegression()
        param_grid = {}

    grid_search = GridSearchCV(model, param_grid, cv=cv_folds, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)

    return grid_search.best_estimator_, grid_search.best_params_, -grid_search.best_score_

def linear_regression_for_tickers(df, model_type='ridge', cv_folds=5):
    results = {}
    
    overall_train_actuals = []
    overall_train_preds = []
    overall_test_actuals = []
    overall_test_preds = []
    
    tickers = df.columns
    for ticker in tickers:
        X_train, X_test, y_train, y_test = prepare_data(df, ticker)
        
        best_model, best_params, best_cv_mae = tune_hyperparameters(X_train, y_train, model_type, cv_folds)
        best_model.fit(X_train, y_train)
        
        y_train_pred = best_model.predict(X_train)
        y_train_pred = np.clip(y_train_pred, a_min=0, a_max=None)  # Ensure no negative values
        mae_train = mean_absolute_error(y_train, y_train_pred)
        r2_train = r2_score(y_train, y_train_pred)
        
        overall_train_actuals.extend(y_train)
        overall_train_preds.extend(y_train_pred)
        
        y_test_pred = best_model.predict(X_test)
        y_test_pred = np.clip(y_test_pred, a_min=0, a_max=None)  # Ensure no negative values
        mae_test = mean_absolute_error(y_test, y_test_pred)
        r2_test = r2_score(y_test, y_test_pred)
        
        overall_test_actuals.extend(y_test)
        overall_test_preds.extend(y_test_pred)
        
        results[ticker] = {
            'model': best_model,
            'best_params': best_params,
            'best_cv_mae': best_cv_mae,  # Best cross-validated MAE
            'train_predictions': y_train_pred,
            'train_mae': mae_train,
            'train_r2': r2_train,
            'test_predictions': y_test_pred,
            'test_mae': mae_test,
            'test_r2': r2_test,
        }
    
    overall_mae_train = mean_absolute_error(overall_train_actuals, overall_train_preds)
    overall_r2_train = r2_score(overall_train_actuals, overall_train_preds)
    overall_mae_test = mean_absolute_error(overall_test_actuals, overall_test_preds)
    overall_r2_test = r2_score(overall_test_actuals, overall_test_preds)
    
    results['overall'] = {
        'train_mae': overall_mae_train,
        'train_r2': overall_r2_train,
        'test_mae': overall_mae_test,
        'test_r2': overall_r2_test,
    }
    
    return results

In [6]:
results = linear_regression_for_tickers(close, model_type='ridge', cv_folds=5)

overall = results['overall']
print(f'Overall Mean Absolute Error (Train set): {overall["train_mae"]}')
print(f'Overall R-squared (Train set): {overall["train_r2"]}')
print(f'Overall Mean Absolute Error (Test set): {overall["test_mae"]}')
print(f'Overall R-squared (Test set): {overall["test_r2"]}')

for ticker, result in results.items():
    if ticker != 'overall':
        print(f'{ticker} Best Params: {result["best_params"]}, Best Cross-Validation MAE: {result["best_cv_mae"]}')

Overall Mean Absolute Error (Train set): 0.006721789588471188
Overall R-squared (Train set): 0.9997868944462591
Overall Mean Absolute Error (Test set): 0.12715592119118063
Overall R-squared (Test set): 0.8518680549035779
A32 Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAA Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: 0.11145013451224654
AAM Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAT Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAV Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABB Best Params: {'alpha': 0.1}, Best Cross-Validation MAE: 1.1884805982065758
ABC Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABI Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABR Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABT Best Params

In [7]:
def complete_predictions(df, results):
    full_predictions_df = pd.DataFrame(index=df.index, columns=df.columns)
    
    for ticker, result in results.items():
        if ticker != 'overall':
            train_indices = df[df.index < '2022-01-01'].index
            test_indices = df[df.index >= '2022-01-01'].index
            
            full_predictions_df.loc[train_indices, ticker] = result['train_predictions']
            full_predictions_df.loc[test_indices, ticker] = result['test_predictions']
    
    return full_predictions_df

In [8]:
close_predictions = complete_predictions(close, results)
close_predictions.shape
close_predictions.to_csv('data/vn_stock/linear_models/ridge_close_top100.txt', sep='\t')

In [9]:
results = linear_regression_for_tickers(open, model_type='ridge', cv_folds=5)

overall = results['overall']
print(f'Overall Mean Absolute Error (Train set): {overall["train_mae"]}')
print(f'Overall R-squared (Train set): {overall["train_r2"]}')
print(f'Overall Mean Absolute Error (Test set): {overall["test_mae"]}')
print(f'Overall R-squared (Test set): {overall["test_r2"]}')

for ticker, result in results.items():
    if ticker != 'overall':
        print(f'{ticker} Best Params: {result["best_params"]}, Best Cross-Validation MAE: {result["best_cv_mae"]}')

open_predictions = complete_predictions(open, results)
print(open_predictions.shape)
open_predictions.to_csv('data/vn_stock/linear_models/ridge_open_top100.txt', sep='\t')

Overall Mean Absolute Error (Train set): 0.007082082049850215
Overall R-squared (Train set): 0.9997762652941028
Overall Mean Absolute Error (Test set): 0.12789672775227823
Overall R-squared (Test set): 0.8519226242821153
A32 Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAA Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: 0.11584702079715656
AAM Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAT Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAV Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABB Best Params: {'alpha': 0.1}, Best Cross-Validation MAE: 1.178871204942543
ABC Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABI Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABR Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABT Best Params:

In [10]:
results = linear_regression_for_tickers(low, model_type='ridge', cv_folds=5)

overall = results['overall']
print(f'Overall Mean Absolute Error (Train set): {overall["train_mae"]}')
print(f'Overall R-squared (Train set): {overall["train_r2"]}')
print(f'Overall Mean Absolute Error (Test set): {overall["test_mae"]}')
print(f'Overall R-squared (Test set): {overall["test_r2"]}')

for ticker, result in results.items():
    if ticker != 'overall':
        print(f'{ticker} Best Params: {result["best_params"]}, Best Cross-Validation MAE: {result["best_cv_mae"]}')

low_predictions = complete_predictions(low, results)
print(low_predictions.shape)
low_predictions.to_csv('data/vn_stock/linear_models/ridge_low_top100.txt', sep='\t')

Overall Mean Absolute Error (Train set): 0.006569044438005514
Overall R-squared (Train set): 0.9997865082490575
Overall Mean Absolute Error (Test set): 0.12484422348840417
Overall R-squared (Test set): 0.8524770505953653
A32 Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAA Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: 0.11088567088759456
AAM Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAT Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAV Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABB Best Params: {'alpha': 0.1}, Best Cross-Validation MAE: 1.1532599187222368
ABC Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABI Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABR Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABT Best Params

In [11]:
results = linear_regression_for_tickers(high, model_type='ridge', cv_folds=5)

overall = results['overall']
print(f'Overall Mean Absolute Error (Train set): {overall["train_mae"]}')
print(f'Overall R-squared (Train set): {overall["train_r2"]}')
print(f'Overall Mean Absolute Error (Test set): {overall["test_mae"]}')
print(f'Overall R-squared (Test set): {overall["test_r2"]}')

for ticker, result in results.items():
    if ticker != 'overall':
        print(f'{ticker} Best Params: {result["best_params"]}, Best Cross-Validation MAE: {result["best_cv_mae"]}')

high_predictions = complete_predictions(high, results)
print(high_predictions.shape)
high_predictions.to_csv('data/vn_stock/linear_models/ridge_high_top100.txt', sep='\t')

Overall Mean Absolute Error (Train set): 0.00667364719917631
Overall R-squared (Train set): 0.9997912674425304
Overall Mean Absolute Error (Test set): 0.12926110680268355
Overall R-squared (Test set): 0.8513126049711736
A32 Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAA Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: 0.1123486000800811
AAM Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAT Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
AAV Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABB Best Params: {'alpha': 0.1}, Best Cross-Validation MAE: 1.2135707553820776
ABC Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABI Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABR Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABS Best Params: {'alpha': 0.001}, Best Cross-Validation MAE: -0.0
ABT Best Params: 